In [ ]:
import flappy_bird_gym
import neat
import cv2
import numpy as np
import pickle
import gym, math
import matplotlib.pyplot as plt
import time

In [ ]:
env = flappy_bird_gym.make("FlappyBird-v0")
config = neat.Config(neat.DefaultGenome, neat.DefaultReproduction, neat.DefaultSpeciesSet, neat.DefaultStagnation, 'config-feedforward.txt')
env.seed(10)
imgarray = []
def eval_genomes(genomes, config):
    for genome_id, genome in genomes:
        ob = env.reset()
        net = neat.nn.recurrent.RecurrentNetwork.create(genome, config)

        current_max_fitness = 0
        fitness_current = 0
        counter = 0
        xpos = 0
        xpos_max = 0

        done = False

        while not done:

            # env.render()
            # time.sleep(.01)

            nnOutput = np.argmax(net.activate((ob[0], ob[1])))
            # ob, reward, done, info = env.step(int(round(max(nnOutput), 0)))
            ob, reward, done, info = env.step(nnOutput)


            xpos = ob[0]
            ypos = ob[1]

            if xpos < 1.66 :
                fitness_current += 1
                xpos_max = xpos

            if round(ypos, 0) == 0:
                fitness_current += 1
            else:
                fitness_current -= 1


            if reward > 1:
                fitness_current += 10
                reward += 1
            
            if done:
                fitness_current -= 10000
                # print(genome_id, fitness_current)

            if fitness_current > current_max_fitness:
                current_max_fitness = fitness_current
                counter = 0
            else:
                counter += 1
            
            if done or counter == 300:
                done = True
                # print(genome_id, fitness_current, counter)
            
            genome.fitness = fitness_current

p = neat.Population(config)

p.add_reporter(neat.StdOutReporter(True))
stats = neat.StatisticsReporter()
p.add_reporter(stats)
# p.add_reporter(neat.Checkpointer(10))

winner = p.run(eval_genomes)

with open('winner.pkl', 'wb') as output:
    pickle.dump(winner, output, 1)



In [ ]:
scorelist = []
def best_model(genome, slowmode=0):
    env = flappy_bird_gym.make("FlappyBird-v0")
    env.seed(10)
    genome = pickle.load(open('winner.pkl', 'rb'))
    ob = env.reset()
    done = False
    net = neat.nn.recurrent.RecurrentNetwork.create(genome, config)

    while not done:
        
        # if slowmode == 1:
        #     time.sleep(0.01)
        #     # env.render()
        # else:
        #     # env.render()

        nnOutput = np.argmax(net.activate((ob[0], ob[1])))
        
        ob, reward, done, info = env.step(nnOutput)

    # print(info['score'])
    scorelist.append(info['score'])
    return scorelist

genome = pickle.load(open('winner.pkl', 'rb'))
for i in range(100):
    scores = (best_model(genome, 0))

print(scores)

In [ ]:
average_scores = sum(scores) / len(scores)
print(average_scores)